# Initializing


## Libraries

In [171]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torchvision import models
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import cv2 as cv
from scipy.ndimage import gaussian_filter
import random
from google.colab import drive
import os




drive.mount('/content/drive')
# %run '/content/drive/MyDrive/Colab Notebooks/CNN_WavePattern_Noise/Fourier_Test/Watermarking_Classes.ipynb'
%run '/content/drive/MyDrive/Colab Notebooks/CNN_WavePattern_Noise/Pre_Trained_Models/MyWatermarkinClasses.ipynb'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## ** Hyper_Parameter

In [172]:
num_train_samples = 1000
img_size = 128

# Mask Ratio
mask_ratio = 0.5


## Augmentation
min_noise_strength = 0.05
max_noise_strength = 2

## Host Image

In [173]:
# folder_path = '/content/drive/My Drive/Colab Notebooks/Image/Host_Image_128'
folder_path = '/content/drive/My Drive/Colab Notebooks/Image/COCO_Gray_Validation_128'

image_array = []
target_size = (128, 128)  # Resize to desired shape (width, height)

for filename in os.listdir(folder_path):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        img_path = os.path.join(folder_path, filename)
        my_image = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
        im = my_image.astype(np.float32)
        im = np.expand_dims(im, axis=-1)
        im = im / 255.0
        im = im[:,:,0]
        image_array.append(im)

# Functions

In [174]:
def created_wave_pattern(image_size, freq):
    freq_variation = 1.5
    x = np.linspace(0, 2 * np.pi, image_size)
    y = np.linspace(0, 2 * np.pi, image_size)
    X, Y = np.meshgrid(x, y)

    # Apply small random frequency variations
    freq_x = freq + np.random.uniform(-freq_variation, freq_variation)
    freq_y = freq + np.random.uniform(-freq_variation, freq_variation)

    # Random phase shifts
    phase_shift_x = np.random.uniform(0, 2 * np.pi)
    phase_shift_y = np.random.uniform(0, 2 * np.pi)

    # Generate wave pattern with varied frequency and phase shift
    wave = np.sin(freq_x * X + phase_shift_x) * np.cos(freq_y * Y + phase_shift_y)

    return normalized_image(wave)


def normalized_image(im):
    # Find the minimum and maximum values in the matrix
    min_value = im.min()
    max_value = im.max()

    # Apply min-max normalization to each element
    normalized_im = (im - min_value) / (max_value - min_value)

    return normalized_im

def Created_Mask(im_size , mask_ratio):
  rows, cols = im_size, im_size
  crow, ccol = rows // 2, cols // 2
  mask = np.zeros((rows, cols), np.uint8)
  mask_radius = int(mask_ratio * im_size)
  cv.circle(mask, (ccol, crow), mask_radius, 1, thickness=-1)
  return mask


def Created_Masked_PN(pn, mask_ratio):

  image_size = pn.shape[0]

  fft_hybrid = np.fft.fft2(pn)
  fft_hybrid_shifted = np.fft.fftshift(fft_hybrid)

  mask = Created_Mask(image_size, mask_ratio)

  high_freqs = fft_hybrid_shifted * (1 - mask)
  reconstructed_pn = np.fft.ifft2(np.fft.ifftshift(high_freqs))
  reconstructed_pn_abs = np.abs(reconstructed_pn)

  epsilon = 1e-8
  spectrum = np.log(reconstructed_pn_abs + epsilon)

  return spectrum

<!-- **bold text**# Augmentation -->

# Evaluation Error

## Calculated Spatial Domain Correlation Detector

In [175]:
# def calculate_spatial_domain_correlation_error_probability(samples_num,
#                                             img_size,
#                                             min_noise,
#                                             max_noise) -> float:

#     error_count = 0

#     for _ in range(samples_num):
#         # Randomly select an image
#         index = np.random.randint(0, len(image_array))
#         image = image_array[index]

#         # Generate pseudo-noise (PN) patterns with varying frequencies
#         frequencies = [5, 10, 15, 20]
#         pn_patterns = [created_wave_pattern(img_size, f) for f in frequencies]

#         # Choose a random watermark pattern
#         watermark_index = np.random.randint(0, len(pn_patterns))

#         # Apply watermarking
#         transmitter = Transmitter(image, pn_patterns[watermark_index], mask_ratio)
#         watermarked_image = transmitter.Watermarking()

#         # Add random noise
#         noise_strength = np.random.uniform(min_noise, max_noise)
#         noisy_watermarked = add_random_noise(watermarked_image, noise_strength)

#         # Correlation-based extraction
#         re = Receiver(noisy_watermarked, mask_ratio)
#         reconstructed_pn = re.PN_Reconstruction()

#         # Created-Masked-PN
#         spatial_masked_pn = np.zeros_like(pn_patterns)

#         for i in range(len(pn_patterns)):
#           re = Receiver(pn_patterns[i], mask_ratio)
#           spatial_masked_pn[i] = re.PN_Reconstruction()


#         correlations = np.array([
#             np.sum(reconstructed_pn * pn) for pn in spatial_masked_pn
#         ])

#         # Identify the most correlated PN pattern
#         detected_index = np.argmax(correlations)

#         # Count false detections
#         if detected_index != watermark_index:
#             error_count += 1

#     # Return accuracy (can be renamed to 'accuracy' or inverted for true error rate)
#     return error_count / samples_num

## Calculated Fourier Domain Correlation Detector

In [203]:
def calculate_fourier_domain_correlation_error_probability(
    samples_num,
    img_size,
    min_noise,
    max_noise) -> float:

    error_count = 0

    for _ in range(samples_num):
        # Randomly select an image
        index = np.random.randint(0, len(image_array))
        image = image_array[index]

        # Generate pseudo-noise (PN) patterns with varying frequencies
        frequencies = [5, 10, 15, 20]
        pn_patterns = [created_wave_pattern(img_size, f) for f in frequencies]

        # Choose a random watermark pattern
        watermark_index = np.random.randint(0, len(pn_patterns))
        # print(f'Watermark index: {watermark_index}')

        # Apply watermarking
        transmitter = Transmitter(image, pn_patterns[watermark_index], mask_ratio)
        watermarked_image = transmitter.Watermarking()

        # Channel Simulation
        channel = Channel(watermarked_image)

        attack_parameter = np.random.uniform(min_noise, max_noise)
        # attack_parameter = np.random.randint(min_noise, max_noise)

        # noisy_watermarked = channel.add_random_noise(attack_parameter)
        # noisy_watermarked = channel.x_shift_attack(attack_parameter)
        # noisy_watermarked = channel.blur_image(attack_parameter)
        # noisy_watermarked = channel.brightness_attack(attack_parameter)
        # noisy_watermarked = channel.contrast_attack(attack_parameter)
        # noisy_watermarked = channel.add_guassian_noise(0 , attack_parameter)
        noisy_watermarked = channel.rotation_attack(attack_parameter)



        # Created Mask
        mask = Created_Mask(img_size, mask_ratio)

        # Reconstruct PN
        fft_hybrid = np.fft.fft2(noisy_watermarked)
        fft_hybrid_shifted = np.fft.fftshift(fft_hybrid)
        reconstructed_pn = fft_hybrid_shifted * (1 - mask)


        # Created-Masked-PN
        fourier_masked_pn = np.zeros_like(pn_patterns)

        for i in range(len(pn_patterns)):
          fft_hybrid = np.fft.fft2(pn_patterns[i])
          fft_hybrid_shifted = np.fft.fftshift(fft_hybrid)
          fourier_masked_pn[i] = fft_hybrid_shifted * (1 - mask)


        # Correlation Detector
        corr = []

        # Flatten both images before correlation
        for i in range(len(fourier_masked_pn)):
            corr_value = np.corrcoef(
                fourier_masked_pn[i].flatten().real,   # ensure real values
                reconstructed_pn.flatten().real
            )[0, 1]
            corr.append(abs(corr_value))

        # Identify the most correlated PN pattern
        detected_index = np.argmax(corr)
        # print(f'Detected index: {detected_index}')

        # Count false detections
        if detected_index != watermark_index:
            error_count += 1

    # Return accuracy (can be renamed to 'accuracy' or inverted for true error rate)
    return error_count / samples_num

# Main

In [205]:
# # Addetive Random Noise
# error_prob = calculate_fourier_domain_correlation_error_probability(10000, img_size, 1.02, 1.04)
# print(f"Error Probability: {error_prob}")


# # Horizontal Shift Attack
# error_prob = calculate_fourier_domain_correlation_error_probability(5000, img_size, 40, 41)
# print(f"Error Probability: {error_prob}")


# # Bluring
# error_prob = calculate_fourier_domain_correlation_error_probability(5000, img_size, 5, 5)
# print(f"Error Probability: {error_prob}")

# # Brightness
# error_prob = calculate_fourier_domain_correlation_error_probability(10000, img_size, 0.5, 0.5)
# print(f"Error Probability: {error_prob}")


# # Contrast
# error_prob = calculate_fourier_domain_correlation_error_probability(5000, img_size, 0.5, 0.5)
# print(f"Error Probability: {error_prob}")

# # Guassian Noise
# error_prob = calculate_fourier_domain_correlation_error_probability(10000, img_size, 0.05, 0.05)
# print(f"Error Probability: {error_prob}")

# Rotation
error_prob = calculate_fourier_domain_correlation_error_probability(5000, img_size, 5, 5)
print(f"Error Probability: {error_prob}")



/tmp/ipython-input-1984594384.py:57: ComplexWarning: Casting complex values to real discards the imaginary part
  fourier_masked_pn[i] = fft_hybrid_shifted * (1 - mask)


Error Probability: 0.7598
